In [ ]:
import base64
import requests
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
from ohmysportsfeedspy import MySportsFeeds
import gmaps
import json

from config import api_key
from config import g_key

msf = MySportsFeeds(version="2.0")
msf.authenticate(api_key, "MYSPORTSFEEDS")

In [ ]:
output = msf.msf_get_data(league='nfl',season='2019-2020-regular',feed='seasonal_player_stats',format='json')
player_list = output['playerStatsTotals']
player_list[0]

In [ ]:
# rb_name = []
# for i in range(0,  len(player_list)):
#     if player_list[i]['player']['primaryPosition'] == 'RB':
#         name = player_list[i]['player']['firstName'] + " " + player_list[i]['player']['lastName']
#         rb_name.append(name)
     
# rb_name

num_records = len(player_list)

stats_df = pd.DataFrame(columns=['Name', 'Position', 'Team', 'Age', 'birthCity', 'College',
                                   'passYards', 'passYPA', 'passTD', 'passInt', 'qbRating',
                                   'rushAttempts', 'rushYards', 'rushAverage', 'rushTD', 
                                   'Targets', 'Receptions', 'recYards', 'recAverage', 'recTD',
                                   'Tackles', 'Sacks', 'TFL', 'Interceptions', 'intTD', 'passesDefended', 'Safeties',
                                   'fumblesForced', 'fumbleTD'], index=range(0, num_records))
stats_df

In [ ]:
for index, row in stats_df.iterrows():
    
    try:
        stats_df.loc[index, 'Name'] = player_list[index]['player']['firstName'] + " " + player_list[index]['player']['lastName']
        stats_df.loc[index, 'Position'] = player_list[index]['player']['primaryPosition']
        stats_df.loc[index, 'Team'] = player_list[index]['team']['abbreviation']
        stats_df.loc[index, 'Age'] = player_list[index]['player']['age']
        stats_df.loc[index, 'birthCity'] = player_list[index]['player']['birthCity']
        stats_df.loc[index, 'College'] = player_list[index]['player']['college']
        stats_df.loc[index, 'passYards'] = player_list[index]['stats']['passing']['passYards']
        stats_df.loc[index, 'passYPA'] = player_list[index]['stats']['passing']['passYardsPerAtt']
        stats_df.loc[index, 'passTD'] = player_list[index]['stats']['passing']['passTD']
        stats_df.loc[index, 'passInt'] = player_list[index]['stats']['passing']['passInt']
        stats_df.loc[index, 'qbRating'] = player_list[index]['stats']['passing']['qbRating']
        stats_df.loc[index, 'rushAttempts'] = player_list[index]['stats']['rushing']['rushAttempts']
        stats_df.loc[index, 'rushYards'] = player_list[index]['stats']['rushing']['rushYards']
        stats_df.loc[index, 'rushAverage'] = player_list[index]['stats']['rushing']['rushAverage']
        stats_df.loc[index, 'rushTD'] = player_list[index]['stats']['rushing']['rushTD']
        stats_df.loc[index, 'Targets'] = player_list[index]['stats']['receiving']['targets']
        stats_df.loc[index, 'Receptions'] = player_list[index]['stats']['receiving']['receptions']
        stats_df.loc[index, 'recYards'] = player_list[index]['stats']['receiving']['recYards']
        stats_df.loc[index, 'recAverage'] = player_list[index]['stats']['receiving']['recAverage']
        stats_df.loc[index, 'recTD'] = player_list[index]['stats']['receiving']['recTD']
        stats_df.loc[index, 'Tackles'] = player_list[index]['stats']['tackles']['tackleTotal']
        stats_df.loc[index, 'Sacks'] = player_list[index]['stats']['tackles']['sacks']
        stats_df.loc[index, 'TFL'] = player_list[index]['stats']['tackles']['tacklesForLoss']
        stats_df.loc[index, 'Interceptions'] = player_list[index]['stats']['interceptions']['interceptions']
        stats_df.loc[index, 'intTD'] = player_list[index]['stats']['interceptions']['intTD']
        stats_df.loc[index, 'passesDefended'] = player_list[index]['stats']['interceptions']['passesDefended']
        stats_df.loc[index, 'Safeties'] = player_list[index]['stats']['interceptions']['safeties']
        stats_df.loc[index, 'fumblesForced'] = player_list[index]['stats']['fumbles']['fumForced']
        stats_df.loc[index, 'fumbleTD'] = player_list[index]['stats']['fumbles']['fumTD']
    
    except:
        pass
        
stats_df

In [ ]:
stats_df['Lat'] = ""
stats_df['Lng'] = ""
stats_df['totalYards'] = stats_df['passYards'] + stats_df['rushYards'] + stats_df['recYards']
stats_df

In [ ]:
city_df = stats_df[stats_df['birthCity'].notna()]
city_df

In [ ]:
gmaps.configure(api_key=g_key)

In [ ]:
for index, row in city_df.iterrows():
    try:
        target_city = city_df.loc[index, 'birthCity']
            
        target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
            'address={0}&key={1}').format(target_city, g_key)

        geo_data = requests.get(target_url).json()

        lat = geo_data["results"][0]["geometry"]["location"]["lat"]
        lng = geo_data["results"][0]["geometry"]["location"]["lng"]

        city_df.loc[index, 'Lat'] = lat
        city_df.loc[index, 'Lng'] = lng
    
    except:
        pass
    
city_df

In [ ]:
locations = city_df[["Lat", "Lng"]]
total_yards = city_df['totalYards']

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=total_yards, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

In [ ]:
qb_df = stats_df[stats_df['Position'] == 'QB']
qb_df = qb_df.sort_values("passYards", ascending=False)
qb_df

In [ ]:
rb_df = stats_df[stats_df['Position'] == 'RB']
rb_df = rb_df.sort_values("rushYards", ascending=False)
rb_df

In [ ]:
wr_df = stats_df.loc[(stats_df['Position'] == 'WR') | (stats_df['Position'] == 'TE')]
wr_df = wr_df.sort_values("recYards", ascending=False)
wr_df